<a href="https://colab.research.google.com/github/Wandergarten/CollectoR/blob/main/%5Bmlverse%5D_PyTorch_implemented_for_R_as_torchvision_torch_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

### `torch`

In [ ]:
devtools::install_github("mlverse/torch")

bit   (NA    -> 4.0.4) [CRAN]
cli   (2.2.0 -> 2.3.0) [CRAN]
coro  (NA    -> 1.0.1) [CRAN]
bit64 (NA    -> 4.0.5) [CRAN]
withr (2.4.0 -> 2.4.1) [CRAN]


Installing 5 packages: bit, cli, coro, bit64, withr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✔  checking for file ‘/tmp/RtmpsZbTeY/remotes3c6e12a33a/mlverse-torch-fcb07d4/DESCRIPTION’
─  preparing ‘torch’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts (502ms)
─  checking for empty or unneeded directories
   Removed empty directory ‘torch/bench’
   Removed empty directory ‘torch/docker’
   Removed empty directory ‘torch/vignettes/examples’
─  building ‘torch_0.2.1.9000.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# a quick check if everything's ok
library(torch)
torch_tensor(1, device = "cuda")

### `torchvision`

In [ ]:
# needed to compile magick (needed by torchvision)
system("sudo apt-get install libmagick++-dev")

In [ ]:
devtools::install_github("mlverse/torchvision")

## Data loading and pre-processing

In [ ]:
library(torchvision)

In [ ]:
dir <- "."

train_ds <- kmnist_dataset(
  dir,
  download = TRUE,
  train = TRUE,
  transform = transform_to_tensor
)

test_ds <- kmnist_dataset(
  dir,
  download = TRUE,
  train = FALSE,
  transform = transform_to_tensor
)



In [ ]:
# the first pair (list) of input and target tensor
train_ds[1]

In [ ]:
# shape of first input tensor
train_ds[1][[1]]$size()

In [ ]:
train_dl <- dataloader(train_ds, batch_size = 32, shuffle = TRUE)
test_dl <- dataloader(test_ds, batch_size = 32)

In [ ]:
train_iter <- train_dl$.iter()
train_iter$.next()

## Network

In [ ]:
net <- nn_module(
  "Net",
  initialize = function() {
    self$conv1 <- nn_conv2d(1, 32, 3)
    self$conv2 <- nn_conv2d(32, 64, 3)
    self$dropout1 <- nn_dropout2d(0.25)
    self$dropout2 <- nn_dropout2d(0.5)
    self$fc1 <- nn_linear(9216, 128)
    self$fc2 <- nn_linear(128, 10)
  },
  forward = function(x) {
    x %>% 
      self$conv1() %>%
      nnf_relu() %>%
      self$conv2() %>%
      nnf_relu() %>%
      nnf_max_pool2d(2) %>%
      self$dropout1() %>%
      torch_flatten(start_dim = 2) %>%
      self$fc1() %>%
      nnf_relu() %>%
      self$dropout2() %>%
      self$fc2()
  }
)

In [ ]:
model <- net()
model$to(device = "cuda")

In [ ]:
optimizer <- optim_adam(model$parameters)

In [ ]:
n_epochs <- 5

for (epoch in 1:n_epochs) {

  l <- c()

  for (b in enumerate(train_dl)) {
    optimizer$zero_grad()
    output <- model(b[[1]]$to(device = "cuda"))
    loss <- nnf_cross_entropy(output, b[[2]]$to(device = "cuda"))
    loss$backward()
    optimizer$step()
    l <- c(l, loss$item())
  }

  cat(sprintf("Loss at epoch %d: %3f\n", epoch, mean(l)))
}

## Evaluate

In [ ]:
model$eval()

In [ ]:
test_losses <- c()
total <- 0
correct <- 0

for (b in enumerate(test_dl)) {
  output <- model(b[[1]]$to(device = "cuda"))
  labels <- b[[2]]$to(device = "cuda")
  loss <- nnf_cross_entropy(output, labels)
  test_losses <- c(test_losses, loss$item())
  predicted <- torch_max(output$data(), dim = 2)[[2]]
  total <- total + labels$size(1)
  correct <- correct + (predicted == labels)$sum()$item()
}


In [ ]:
mean(test_losses)

In [ ]:
test_accuracy <-  correct/total
test_accuracy